In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
# I found a bug: sometimes, I get back the "arxiv fulltext PDF or arxiv snapshot, not the entry itself."
# I think the fix is to individually embed just the entries that are not PDFs as JSON and then query them.

In [ ]:
from llamabot.zotero.library import ZoteroLibrary
from pathlib import Path

ZOTERO_JSON_DIR = Path.home() / ".llamabot/zotero/zotero_index/"
ZOTERO_JSON_DIR.mkdir(parents=True, exist_ok=True)

# library = ZoteroLibrary()

In [ ]:
from llamabot import QueryBot
from llamabot.prompt_library.zotero import get_key, retrieverbot_sysprompt


retrieverbot = QueryBot(
    retrieverbot_sysprompt(),
    doc_paths=list(ZOTERO_JSON_DIR.glob("*.json")),
    stream=True,
    use_cache=False,
)

In [ ]:
retriever = retrieverbot.index.as_query_engine(similarity_topk=10)

retriever.retrieve("bayesian optimization")

In [ ]:
retrieverbot(
    "I'm looking for a paper on Bayesian optimization. Which one should I read?"
)

In [ ]:
for k, v in library.library.items():
    if v.has_pdf():
        print(k)

In [ ]:
asdfasdfasdfasdf

In [ ]:
from llamabot import QueryBot

In [ ]:
from llamabot.prompt_library.zotero import retrieverbot_sysprompt, get_key
from pathlib import Path
from llamabot import QueryBot

ZOTERO_JSON_PATH = Path.home() / ".llamabot/zotero/zotero_index.json"

retrieverbot = QueryBot(
    retrieverbot_sysprompt(),
    doc_paths=[ZOTERO_JSON_PATH],
)

In [ ]:
# CLI: llamabot zotero configure --library-id 12345 --library-type "user" --api-key 1p84325f

# Then we store the library ID and the library type in the config file.

In [ ]:
retrieverbot(get_key("A paper on machine learning for engineering GFP"))

In [ ]:
import json
from llamabot.zotero.library import ZoteroLibrary


lib = ZoteroLibrary()
lib["BV68IW7P"].download_pdf(Path("/tmp"))

In [ ]:
# llamabot zotero sync

In [ ]:
from pyzotero import zotero
import os
from dotenv import load_dotenv

load_dotenv()

zotero_library_id = os.environ.get("ZOTERO_LIBRARY_ID", None)
zotero_library_type = os.environ.get("ZOTERO_LIBRARY_TYPE", None)
zotero_api_key = os.environ.get("ZOTERO_API_KEY", None)

zot = zotero.Zotero(
    library_id=zotero_library_id,
    library_type=zotero_library_type,
    api_key=zotero_api_key,
)
items = zot.everything(zot.items())

In [ ]:
len(items)

In [ ]:
# We only want the parent items. They don't have a "parent" key.

# items[3]
# [item for item in items if "up" not in item["links"]]
items_with_abstracts = [
    item for item in items if "data" in item and "abstractNote" in item["data"]
]
len(items_with_abstracts)

In [ ]:
# QueryBot

# from llamabot import QueryBot
title = items_with_abstracts[0]["data"]["title"]
abstract = items_with_abstracts[0]["data"]["abstractNote"]
authors = [
    c
    for c in items_with_abstracts[0]["data"]["creators"]
    if c["creatorType"] == "author"
]
items_with_abstracts[0]

Save all of the JSONS to disk under /tmp/zotero_jsons


In [ ]:
# Desired CLI:
# llamabot zotero chat --title "Title" --author "any author"

# Step 1: Retrieve the PDF key from Zotero. Raise error if there is no PDF key.

In [ ]:
from llamabot.prompt_library.zotero import retrieverbot_sysprompt
from pathlib import Path
from llamabot import QueryBot

ZOTERO_JSON_PATH = Path.home() / ".llamabot/zotero/zotero_index.json"

retrieverbot = QueryBot(
    retrieverbot_sysprompt(),
    doc_paths=[ZOTERO_JSON_PATH],
)

In [ ]:
from llamabot.prompt_library.zotero import get_key

response = retrieverbot(
    get_key(title="A Connection Between Score Matching and Denoising Autoencoders")
)

In [ ]:
import json
from llamabot.zotero.utils import load_zotero
from llamabot.zotero.library import ZoteroLibrary, ZoteroItem

key = json.loads(response.content)["key"]


zot = load_zotero()

# library = zot.everything(zot.items())
library = ZoteroLibrary(ZOTERO_JSON_PATH)
fpath = library[key].download_pdf(Path("/tmp"))
fpath

In [ ]:
library[key]["data.title"]

In [ ]:
docchat = QueryBot(
    system_message="You are an expert paper reader.",
    doc_paths=[fpath],
)

In [ ]:
docchat("Summarize this paper for me.")

In [ ]:
docchat("What implementation of Gaussian Processes did the author use?")

In [ ]:
docchat("What is the reference for COMBO? Is there a paper?")

In [ ]:
docchat("How big was the initial small library that they tested?")

In [ ]:
docchat("Define 'point saturation mutagenesis' for me please.")

In [ ]:
len(items)

items[1]["links"]["attachment"]

In [ ]:
items[0]["links"]["attachment"]["href"]

In [ ]:
with open("/tmp/article.pdf", "wb") as f:
    f.write(zot.file("A4BK56EA"))

In [ ]:
# We can parse the href to get the item key

# I think we need to